<a href="https://colab.research.google.com/github/jameschan0/AI-Project/blob/main/%E5%8D%B7%E7%A9%8D%E7%A5%9E%E7%B6%93%E7%B6%B2%E8%B7%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
# hyperparamter
BATCH_SIZE = 128
num_classes = 10
epochs = 10
learning_rate = 1e-4 # 0.0001
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = BATCH_SIZE, shuffle = False)
import numpy as np
import matplotlib.pyplot as plt
iterator = iter(train_loader)
images, labels = next(iterator)
fig, axes = plt.subplots(1,20, figsize=(15, 15))
for i in range(20):
  axes[i].imshow(images[i][0], cmap = 'gray')
  axes[i].set_axis_off()
fig.show()

In [ ]:
class Convnet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=1,padding=1)#16,28,28
    self.conv2=nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,stride=1,padding=1)
    self.pool1=nn.MaxPool2d(2,2)#16,14,14
    self.conv3=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1)#32,14,14
    self.conv4=nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1)
    self.pool2=nn.MaxPool2d(2,2)#32,7,7
    self.fc1=nn.Linear(32*7*7,10)
  def forward(self, x):
    x1=F.relu(self.conv1(x))
    x2=F.relu(self.conv2(x1))
    x3=self.pool1(x2)
    x4=F.relu(self.conv3(x3))
    x5=F.relu(self.conv4(x4))
    x6=self.pool2(x5)
    x6=x6.view(-1,32*7*7)
    output=self.fc1(x6)
    return output

In [ ]:
cnn = Convnet().to(device)
print(cnn)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [ ]:
# training loop
total_steps = len(train_loader)
for epoch in range(epochs):
  for i, (images, labels) in enumerate(train_loader):
    # batch.size 1 28 28
    images = images.to(device)
    labels = labels.to(device)

    # forward
    outputs = cnn(images)
    loss = criterion(outputs, labels)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print (f'Epoch {epoch+1}, Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')

In [ ]:
# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = cnn(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')
